In [1]:
import cv2
from utils import model
from utils import data_loader
import tensorflow as tf

import numpy as np

from tqdm import tqdm
from skimage.transform import resize

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from utils.heat_map import HeatMap
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2023-02-26 06:07:55.123088: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/lbbduy/anaconda3/envs/master_python/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-26 06:07:55.885901: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-02-26 06:07:55.886447: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-02-26 06:07:55.896256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-26 06:07:55.896648: I tensorflow/core/common_runtime/gpu/gpu_device.cc:172

In [44]:
model_pretrained = model.load_pretrained_model('./model_checkpoints/model_WNet_NoBN_checkpoint_12_05_2022_013501.h5')
# model_pretrained = model.load_pretrained_model('./model_checkpoints/model_CSRNet_checkpoint_12_03_2022_233004.h5')
# model_pretrained = model.load_pretrained_model('./model_checkpoints/model_U-ASD-Net_MSE_BCE_Loss_checkpoint_02_16_2023_000048.h5')

OUTPUT_RATIO = 2
WEIGHT = 0.6

In [46]:
def norm_image(img):
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    return img

def opencv_plot(densmap_name):

    densmap = np.array(densmap_name, np.float32)
    densmap = cv2.resize(densmap, (int(densmap.shape[1]*OUTPUT_RATIO), int(densmap.shape[0]*OUTPUT_RATIO)))
    densmap = cv2.blur(densmap, (3, 3))

    densmap[densmap >1 ] = 1
    densmap = norm_image(densmap) * 255
    densmap = densmap.astype(np.uint8)

    hm = cv2.applyColorMap(densmap, cv2.COLORMAP_JET)

    return hm

In [49]:
cap = cv2.VideoCapture('/home/lbbduy/Documents/Master_Final/examples/videos_demo/hanoi-traffic-in-rain.mp4')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
cap.set(cv2.CAP_PROP_FPS, 60.0)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, int(cap.get(3)))
cap.set(cv2.CAP_PROP_FRAME_WIDTH,int(cap.get(4)))
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if int(major_ver) < 3:
    fps = cap.get(cv2.cv.CV_CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
else:
    fps = cap.get(cv2.CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

# Check if camera opened successfully
if (cap.isOpened()== False):
  print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret:
    image = cv2.resize(frame, (640, 480))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = tf.image.convert_image_dtype(image, tf.dtypes.float32)
    image = tf.expand_dims(image , 0)

    dm = model_pretrained.predict(image)[0]
    count  = np.sum(dm)
    dm = opencv_plot(dm)

    frame = cv2.resize(frame,(dm.shape[1], dm.shape[0]))
    hm = cv2.addWeighted(dm, WEIGHT, frame , 1 - WEIGHT, 0, dm)

    # Display the resulting frame

    fontface = cv2.FONT_HERSHEY_DUPLEX
    fontscale = 1
    fontcolor = (255, 255, 255)
    cv2.putText(hm, "Count : "+ str(int(count)+1), (50,50), fontface, fontscale, fontcolor)
    cv2.imshow('Frame',hm)


    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else:
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.0
